# Time Series Anomaly Detection using LSTM Autoencoders with PyTorch in Python

In [2]:
!nvidia-smi


Wed May 21 13:44:44 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 4500 Ada Gene...    Off | 00000000:21:00.0 Off |                  Off |
| 45%   69C    P2             170W / 210W |  24107MiB / 24570MiB |     99%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
## !pip install -qq arff2pandas

In [4]:
!pip install -q -U watermark

In [5]:
!pip install -qq -U pandas

In [6]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,arff2pandas

Python implementation: CPython
Python version       : 3.9.20
IPython version      : 8.15.0

numpy      : 2.0.2
pandas     : 2.2.3
torch      : 2.7.0
arff2pandas: not installed



In [7]:
pip install --upgrade torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install --upgrade numpy

In [9]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

from torch import nn, optim

import torch.nn.functional as F
import csv



%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

#RANDOM_SEED = 42

Create function to set random seed for randomness splitting data and randomness initial of the model

In [10]:
def set_seed( RANDOM_SEED):
    np.random.seed(RANDOM_SEED)
    torch.manual_seed(RANDOM_SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(RANDOM_SEED)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # using GPU for faster computation if Cuda is available

In [12]:
!pip install gdown

In [13]:
!gdown --id 16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT

/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT
To: /home/lamhuutoan.nguyen/smd_project/SMD-Autoencoder/ECG5000.zip
100%|██████████████████████████████████████| 10.6M/10.6M [00:00<00:00, 14.5MB/s]


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # using GPU for faster computation if Cuda is available

In [15]:
#smd_train_df = get_data("OmniAnomaly/ServerMachineDataset/train/")
#smd_test_df = get_data("OmniAnomaly/ServerMachineDataset/test/")
#labels_df = get_data("OmniAnomaly/ServerMachineDataset/test_label/")

In [16]:
machine_1_train = pd.read_csv("OmniAnomaly/ServerMachineDataset/train/machine-1-1.txt", header=None, dtype={0: str})
machine_1_train = machine_1_train.replace(r'[^\d.-]', '', regex=True)

In [17]:
machine_1_test = pd.read_csv("OmniAnomaly/ServerMachineDataset/test/machine-1-1.txt", header=None, dtype={0: str})
machine_1_test = machine_1_test.replace(r'[^\d.-]', '', regex=True)

In [18]:
machine_1_test_label = pd.read_csv("OmniAnomaly/ServerMachineDataset/test_label/machine-1-1.txt", header=None, dtype={0: str})
machine_1_test_label = machine_1_test_label.replace(r'[^\d.-]', '', regex=True)
machine_1_test_label = pd.to_numeric(machine_1_test_label.iloc[:, 0], errors='coerce')

In [19]:
type(machine_1_test_label)

pandas.core.series.Series

In [20]:
def train_val_split(RANDOM_SEED, train):
    train_df, val_df = train_test_split(
    train,
    test_size=0.2,
    random_state=RANDOM_SEED)
    return train_df, val_df

In [21]:

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, default_collate
class CustomeDataSet(Dataset):  # Custom dataset used for train- and val-dataset
  
    def __init__(self,data):
        data = data.astype(np.float32).to_numpy()
        self.dataset = [torch.tensor(s).unsqueeze(1).float() for s in data]     
    # support indexing such that dataset[i] can  
    # be used to get i-th sample 
    def __getitem__(self, index): 
        return self.dataset[index] 
        
    # we can call len(dataset) to return the size 
    def __len__(self): 
        return len(self.dataset) 
    def getShape(self):
    
        n_features = self.dataset[0].shape[1]
        seq_len = self.dataset[0].shape[0]
        return seq_len, n_features

In [22]:
class TestDataSet(Dataset): # use for test-set cause we want to also modify the test-label to match the test_set for calculating the AUC
  
    def __init__(self,data,target):
        data = data.astype(np.float32).to_numpy()
        self.dataset = [torch.tensor(s).unsqueeze(1).float() for s in data]
        self.target = target
    # support indexing such that dataset[i] can  
    # be used to get i-th sample 
    def __getitem__(self, index): 
        current_data = self.dataset[index]
        current_target = self.target[index]
        return {
            "x" : current_data,
            "y" : current_target
        }
        
    # we can call len(dataset) to return the size 
    def __len__(self): 
        return len(self.dataset) 
    def getShape(self):
    
        n_features = self.dataset[0].shape[1]
        seq_len = self.dataset[0].shape[0]
        return seq_len, n_features

In [23]:
def train_val_test_Loader_create(train_df, val_df, test_df, test_target_df):
    train_dataset = CustomeDataSet(train_df)
    val_dataset = CustomeDataSet(val_df)
    test_dataset = TestDataSet(test_df,test_target_df)
    train_Loader = DataLoader(train_dataset, shuffle=True, batch_size = 10)
    val_Loader = DataLoader(val_dataset, shuffle=True, batch_size = 2)
    test_Loader = DataLoader(test_dataset, shuffle=False, batch_size = 1)
    #test_Loader = create_test_dataset(test_df)
    seq_len , n_features =  train_dataset.getShape()
    return train_Loader, seq_len, n_features, val_Loader, test_Loader

Each Time Series will be converted to a 2D Tensor in the shape *sequence length* x *number of features* (140x1 in our case).

Let's create some datasets:

In [24]:
class Encoder(nn.Module):

  def __init__(self, seq_len, n_features, rnn_type, number_layers, embedding_dim=32, n_stages =2):
    super(Encoder, self).__init__()

    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
    self.rnn_type = rnn_type
    self.n_stages = n_stages
    self.number_layers = number_layers
    rnn_dict = {
            'LSTM': nn.LSTM,
            'GRU': nn.GRU
        }

    self.rnn_stages = nn.ModuleList() # a list to hold RNN_stages but not the last stage
    if self.n_stages ==1:
        self.output_stage = rnn_dict[self.rnn_type.upper()]( # the last stage
          input_size=self.n_features,
          hidden_size= self.embedding_dim,
          num_layers= self.number_layers,
          batch_first=True
        )
    else:
        # input_size > hidden_size, cause in Encoder we want to compress the data, reduce the dimension into a  
        # representation, which can capture the most significant features and will be passed to the decoder
        self.rnn_stages.append(  # append the first stage cause input_size = n_features
          rnn_dict[self.rnn_type.upper()](
            input_size=self.n_features,
            hidden_size=self.hidden_dim,
            num_layers= self.number_layers,
            batch_first=True
          )
        )
        
        
        for i in range(1,self.n_stages-1):# append the rest stages except the last one
            self.rnn_stages.append(
            rnn_dict[self.rnn_type.upper()](
                input_size=self.hidden_dim, # diff from the first stage
                hidden_size=self.hidden_dim,
                num_layers= self.number_layers,
                batch_first=True
              )
            )
        
        self.output_stage = rnn_dict[self.rnn_type.upper()]( # the last stage
          input_size=self.hidden_dim,
          hidden_size= self.embedding_dim,
          num_layers= self.number_layers,
          batch_first=True
        )



  def forward(self, x):
    if x.size(0) ==1: #(checking if the batch_num equal 1)
        x = x.reshape((1, self.seq_len, self.n_features))
    else :
        x = x.reshape((x.size(0), self.seq_len, self.n_features)) #(cause batch is set by DataLoader)
    if self.rnn_type.upper() == "GRU" :
      for stage in self.rnn_stages:
        x,_ = stage(x)
      x,hidden_n = self.output_stage(x)

      # x, _,  = self.rnn1(x) # x, hidden_n
      # x, hidden_n  = self.rnn2(x) # x, hidden_n
    else:
      for stage in self.rnn_stages:
        x,(_,_) = stage(x)
      x,(hidden_n,_) = self.output_stage(x)

      # x, (_,_)  = self.rnn1(x) # x, hidden_n
      # x, (hidden_n,_)  = self.rnn2(x) # x, hidden_n
    hidden_n = hidden_n[-1]
    if x.size(0) ==1: #(checking if the batch_num equal 1)
        return hidden_n.reshape((self.n_features, self.embedding_dim))
    else:
        return hidden_n.reshape((x.size(0), self.embedding_dim)) #( dimension of ( batch, embbeding_dim))

The *Encoder* uses two LSTM layers to compress the Time Series data input.

Next, we'll decode the compressed representation using a *Decoder*:

In [25]:
class Decoder(nn.Module):

  def __init__(self, seq_len, rnn_type, number_layers, input_dim=32, n_features=1, n_stages =2 ):
    super(Decoder, self).__init__()

    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2*input_dim, n_features
    self.rnn_type = rnn_type
    self.n_stages = n_stages
    self.number_layers = number_layers  


    rnn_dict = {
            'LSTM': nn.LSTM,
            'GRU': nn.GRU
        }
    # input_size < hidden_size cause we want to reconstruct the original input from the passed representation
    # of the encoder
    self.rnn_stages = nn.ModuleList()
    for _ in range(self.n_stages -1 ):
        self.rnn_stages.append(
          rnn_dict[self.rnn_type.upper()]( #GRU
              input_size=input_dim,
              hidden_size=input_dim,
              num_layers=self.number_layers,
              batch_first=True
            )
        )

    self.rnn_stages.append(rnn_dict[self.rnn_type.upper()]( #GRU
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=self.number_layers,
      batch_first=True
    ))

    self.output_layer = nn.Linear(self.hidden_dim, self.n_features) # last stage

  def forward(self, x):
    batch_size = x.size(0)
    if x.size(0) ==1: #(checking if the batch_num equal 1)
        x = x.repeat(self.seq_len, self.n_features)
        x = x.reshape((self.n_features, self.seq_len, self.input_dim))
    else:
        x = x.repeat(1,self.seq_len, 1)
        x = x.reshape((batch_size, self.seq_len, self.input_dim))
    if self.rnn_type.upper() == "GRU" :
        for stage in self.rnn_stages:
            x, _ = stage(x)
        # x, _ = self.rnn1(x) # x, hidden_n
        # x, _ = self.rnn2(x) # x, hidden_n
    else:
        for stage in self.rnn_stages:
            x, (_,_) = stage(x)
        #x, (_, _) = self.rnn1(x)
        # x, (_, _) = self.rnn2(x)
    if x.size(0) ==1: #(checking if the batch_num equal 1)
        x = x.reshape((self.seq_len, self.hidden_dim))
    else:
        x = x.reshape((batch_size,self.seq_len, self.hidden_dim))

    return self.output_layer(x)

Our Decoder contains two LSTM layers and an output layer that gives the final reconstruction.

Time to wrap everything into an easy to use module:

In [26]:
class RecurrentAutoencoder(nn.Module):

  def __init__(self, seq_len, n_features, number_layers,type, n_stages, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()

    self.encoder = Encoder(seq_len, n_features,type,number_layers, embedding_dim, n_stages).to(device)
    self.decoder = Decoder(seq_len, type, number_layers, embedding_dim, n_features, n_stages).to(device)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

Our Autoencoder passes the input through the Encoder and Decoder. Let's create an instance of it:

In [27]:
def create_model(type,seq_len, n_features, number_layers, n_stages): # we can create a LSTM-autoencoder or GRU-autoencoder
  model = RecurrentAutoencoder(seq_len, n_features, number_layers, type, n_stages, 128)
  model = model.to(device)
  return model

## Training

Let's write a helper function for our training process:

In [28]:
def train_model(model, train_Loader, val_Loader, n_epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
  criterion = nn.L1Loss(reduction='sum').to(device)  
  #history = dict(train=[], val=[])
  #patience = 5



  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10000.0

  for epoch in range(1, n_epochs + 1):
    model = model.train()
    #num_train = 0
    #num_val = 0
    train_losses = []
    for batch_true in train_Loader:
      #num_train += 1
      optimizer.zero_grad()

      batch_true = batch_true.to(device)
      batch_pred = model(batch_true)

      loss = criterion(batch_pred, batch_true)

      loss.backward()
      optimizer.step()

      train_losses.append(loss.item())

    val_losses = []
    model = model.eval()
    with torch.no_grad():
      for batch_true in val_Loader:
        #num_val +=1

        batch_true = batch_true.to(device)
        batch_pred = model(batch_true) # feed the model with the input (batch_true) to get the prediction(batch_predict)

        loss = criterion(batch_pred, batch_true) # calculate the diff between the input(batch_true)
                                            # and the predict of the model ( batch_pred)
        val_losses.append(loss.item())

    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)

    #history['train'].append(train_loss)
    #history['val'].append(val_loss)

    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
      #patience =5
    #else:
      #patience -= 1
      #if patience == 0:
          #print(f"break at epoch {epoch}")
          #break
    #print(f"num_train : {num_train}")
    #print(f"num_val : {num_val}")

    

  model.load_state_dict(best_model_wts)
  return model.eval() #, history

At each epoch, the training process feeds our model with all training examples and evaluates the performance on the validation set. Note that we're using a batch size of 1 (our model sees only 1 sequence at a time). We also record the training and validation set losses during the process.

Note that we're minimizing the [L1Loss](https://pytorch.org/docs/stable/nn.html#l1loss), which measures the MAE (mean absolute error). Why? The reconstructions seem to be better than with MSE (mean squared error).

We'll get the version of the model with the smallest validation error. Let's do some training:

In [29]:
def trained_model(model, train_Loader, val_Loader, n_epochs):
  model = train_model(
    model,
    train_Loader,
    val_Loader,
    n_epochs
  )
  return model


## Saving the model

Let's store the model for later use:

In [30]:
#MODEL_PATH = 'model.pth'

#torch.save(model, MODEL_PATH)

Uncomment the next lines, if you want to download and load the pre-trained model:

In [31]:
# !gdown --id 1jEYx5wGsb7Ix8cZAw3l5p5pOwHs3_I9A
# model = torch.load('model.pth')
# model = model.to(device)

## Choosing a threshold

With our model at hand, we can have a look at the reconstruction error on the training set. Let's start by writing a helper function to get predictions from our model:

In [32]:
def predict(model, data_Loader):
    predictions, losses, targets = [], [], []
    criterion = nn.L1Loss(reduction='sum').to(device)  # Use 'none' to get per-sample loss
 
    with torch.no_grad():
        model = model.eval()
        for data in data_Loader:
            batch_true = data["x"]
            target = data["y"]
            print(type(target))
            
            batch_true = batch_true.to(device)
            target = target.to(device)

            batch_pred = model(batch_true)
            loss = criterion(batch_pred, batch_true)  # Loss per batch
            
            predictions.append(batch_pred.cpu().numpy().flatten())
            losses.append(loss.item())  # Store one loss value per batch
            targets.append(target.cpu().numpy().flatten())  # Aggregate targets per batch  

    return predictions, losses, targets 

Our function goes through each example in the dataset and records the predictions and losses. Let's get the losses and have a look at them:

Calculate the AUC_SCORE

In [33]:
def cal_AUC (target_data, pred_losses):
  target_data = np.array(target_data, dtype=int)
  target_data = target_data.flatten()
  arg = np.argsort(pred_losses) # arg[i] show the i+1 smallest lossvalue is pred_losses[arg[i]]
     # show that rank[i] is the rank of pred_losses[i]
  indices = np.arange(len(target_data))[target_data == 1]
  #print("target_data shape:", np.shape(target_data))
  n1 = len(indices)
  n2 = len(target_data) - n1
  #print( f" n1: {n1}")
  #print( f" n2: {n2}")
  #print(f" rank: {rank}")
  #print(f" arg: {arg}")
  #print("target shape")
  #print(len(target_data))
  #print(len(target_data[0]))  
  #print("losses shape")
  #print(len(pred_losses))
  #print(pred_losses[0])  
  
  
  if n1 ==0 or n2 ==0:
    print("fail")
    raise Exception("Unable to calculate AUC score. Only elements of one class present.")
    #return 0

  return (np.mean(rank[indices]) - (n1 +1)/2)/ n2

## Now we will write some scripts to run our defined funtions above

Creating the dataset with different seed 

In [34]:
def create_data(seed):

    RANDOM_SEED = set_seed(seed)
    train_df, val_df = train_val_split(RANDOM_SEED, machine_1_train) 
    train_Loader, seq_len, n_features, val_Loader, test_Loader  = train_val_test_Loader_create(train_df, val_df, machine_1_test, machine_1_test_label)
  
    return train_Loader, seq_len, n_features, val_Loader, test_Loader


Get the model after training 

In [35]:
def get_model(type,seq_len,n_features, train_Loader, val_Loader,n_epochs,number_layers, n_stages):
    model = create_model(type,seq_len, n_features,number_layers, n_stages) # create model 
    model = trained_model(model, train_Loader, val_Loader,n_epochs) # train model
    return model    

Caculate the AUC for each model and dataset, also save the Reconstruction Errors

In [36]:
def AUC_and_predLoss(type,seq_len,n_features, train_Loader, val_Loader, test_Loader, n_epochs, number_layers, n_stages):
  model = get_model(type,seq_len,n_features, train_Loader, val_Loader, n_epochs,number_layers, n_stages)
  _, pred_losses, targets = predict(model, test_Loader)
  #Reconstruction_Error["GRU"] = pred_losses
  AUC_score = cal_AUC(targets, pred_losses)
  #AUC["GRU"].append(AUC_GRU_score)
  return pred_losses, AUC_score

Here we will create n_models ( n GRU-models and n LSTM-models), and train each with n_epochs and each will have n_stages in Encoder and Decoder. So we will have n models of each type with different initial parameter base on Seed and the dataset, which we use to train, validation and test our models, is also split in different ways based on Seed. Thanks to that, we get the average performance results of each type of model

In [37]:
def AUC_df(n_epochs, n_models, number_layers, n_stages,file_name, model_type):
  AUC = []
  Reconstruction_Error= []
  for i in range(n_models): # create n_models of GRU, LTSM and calculate AUC to evaluate the perfomences of these 2
    train_Loader, seq_len, n_features, val_Loader, test_Loader = create_data(i*5)
    ## FIRST the GRU
    pred_losses, AUC_score = AUC_and_predLoss(model_type,seq_len,n_features, train_Loader, val_Loader, test_Loader, n_epochs, number_layers, n_stages)
    Reconstruction_Error = pred_losses
    AUC.append(AUC_score)

    AUC_df = pd.DataFrame(AUC)
    Reconstruction_Error_df = pd.DataFrame(Reconstruction_Error)
    with open(file_name, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([i, AUC_score]) 
  return AUC_df,Reconstruction_Error_df


plot a boxplot to show the AUC of LSTM and GRU models, so we can compare the performance of these 2 in general but not for a specific initial parameter or specific dataset

In [38]:
def plot_boxplot (auc):

  auc_melted = auc.melt(var_name='Model', value_name='AUC')
  sns.boxplot(x='Model', y='AUC', data=auc_melted)
  plt.title('AUC Scores Comparison Across Models')
  plt.show()

In [39]:
def load_auc(path):
    auc = pd.read_csv(path, header = None)
    auc = auc.iloc[:,1:3]
    auc.columns =  ["GRU", "LSTM"]
    return auc

In [40]:
from datetime import datetime
def duration_auc_reError(n_epochs, n_models, number_layers, n_stages, result_file_name, runtime_file_name,model_type):
    start_time = datetime.now()
    if model_type == "GRU":
        auc, Reconstruction_Error = AUC_df(n_epochs, n_models, number_layers, n_stages,result_file_name, model_type)
    else:
        auc, Reconstruction_Error = AUC_df(n_epochs, n_models, number_layers, n_stages,result_file_name, model_type)
    end_time = datetime.now()
    duration = end_time - start_time
    with open(runtime_file_name, 'a', newline='') as file:
            file.write(f"{str(duration)}\n")
    return duration, auc, Reconstruction_Error

This function will return the runtime 

In [41]:
duration_1, auc_1, Reconstruction_Error_1 = duration_auc_reError(50, 10, 3, 1,"GRU_test_one_Stages_three_Layer_testab.csv","GRU_Runtime_1_3_testab.csv","GRU")


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [41]:
duration_1, auc_1, Reconstruction_Error_1 = duration_auc_reError(50, 10, 3, 1,"LSTM_test_one_Stages_three_Layer.csv","LSTM_runtime_1_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [40]:
duration_2_GRU, auc_2_GRU, Reconstruction_Error_2_GRU = duration_auc_reError(50, 10, 3, 2,"GRU_test_two_Stages_three_layer.csv","GRU_runtime_2_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

chay 3 layer toi day


In [43]:
duration_2_LSTM, auc_2_LSTM, Reconstruction_Error_2_LSTM = duration_auc_reError(50, 10, 3, 2,"LSTM_test_two_Stages_three_layer.csv","LSTM_runtime_2_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [44]:
duration_3_GRU, auc_3_GRU, Reconstruction_Error_3_GRU = duration_auc_reError(50, 10, 3, 3,"GRU_test_three_Stages_three_layer.csv","GRU_runtime_3_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [45]:
duration_3_LSTM, auc_3_LSTM, Reconstruction_Error_3_LSTM = duration_auc_reError(50, 10, 3, 3,"LSTM_test_three_Stages_three_layer.csv","LSTM_runtime_3_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [ ]:
duration_4_GRU, auc_4_GRU, Reconstruction_Error_4_GRU = duration_auc_reError(50, 10, 3, 4,"GRU_test_four_Stages_three_layer.csv","GRU_runtime_4_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [39]:
duration_4_LSTM, auc_4_LSTM, Reconstruction_Error_4_LSTM = duration_auc_reError(50, 10, 3, 4,"LSTM_test_four_Stages_three_layer.csv","LSTM_runtime_4_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [40]:
duration_5_GRU, auc_5_GRU, Reconstruction_Error_5_GRU = duration_auc_reError(50, 10, 3, 5,"GRU_test_five_Stages_three_layer.csv","GRU_runtime_5_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [41]:
duration_5_LSTM, auc_5_LSTM, Reconstruction_Error_5_LSTM = duration_auc_reError(50, 10, 3, 5,"LSTM_test_five_Stages_three_layer.csv","LSTM_runtime_5_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [42]:
duration_6_GRU, auc_6_GRU, Reconstruction_Error_6_GRU = duration_auc_reError(50, 10, 3, 6,"GRU_test_six_Stages_three_layer.csv","GRU_runtime_6_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [43]:
duration_6_LSTM, auc_6_LSTM, Reconstruction_Error_6_LSTM = duration_auc_reError(50, 10, 3, 6,"LSTM_test_six_Stages_three_layer.csv","LSTM_runtime_6_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [44]:
duration_7_GRU, auc_7_GRU, Reconstruction_Error_7_GRU = duration_auc_reError(50, 10, 3, 7,"GRU_test_seven_Stages_three_layer.csv","GRU_runtime_7_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [45]:
duration_7_LSTM, auc_7_LSTM, Reconstruction_Error_7_LSTM = duration_auc_reError(50, 10, 3, 7,"LSTM_test_seven_Stages_three_Layer.csv","LSTM_runtime_7_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [46]:
duration_8_GRU, auc_8_GRU, Reconstruction_Error_8_GRU = duration_auc_reError(50, 10, 3, 8,"GRU_test_eight_Stages_three_Layer.csv","GRU_runtime_8_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [47]:
duration_8_LSTM, auc_8_LSTM, Reconstruction_Error_8_LSTM = duration_auc_reError(50, 10, 3, 8,"LSTM_test_eight_Stages_three_Layer.csv","LSTM_runtime_8_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [39]:
duration_9_GRU, auc_9_GRU, Reconstruction_Error_9_GRU = duration_auc_reError(50, 10, 3, 9,"GRU_test_nine_Stages_three_Layer.csv","GRU_runtime_9_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [39]:
duration_9_LSTM, auc_9_LSTM, Reconstruction_Error_9_LSTM = duration_auc_reError(50, 10, 3, 9,"LSTM_test_nine_Stages_three_Layer.csv","LSTM_runtime_9_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [40]:
duration_10_GRU, auc_10_GRU, Reconstruction_Error_10_GRU = duration_auc_reError(50, 10, 3, 10,"GRU_test_ten_Stages_three_Layer.csv","GRU_runtime_10_3.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [41]:
duration_10_LSTM, auc_10_LSTM, Reconstruction_Error_10_LSTM = duration_auc_reError(50, 10, 3, 10,"LSTM_test_ten_Stages_three_Layer.csv","LSTM_runtime_10_3.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

chay cho 2

In [42]:
duration_10_GRU, auc_10_GRU, Reconstruction_Error_10_GRU = duration_auc_reError(50, 10, 2, 10,"GRU_test_ten_Stages_two_Layer.csv","GRU_runtime_10_2.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [ ]:
duration_10_LSTM, auc_10_LSTM, Reconstruction_Error_10_LSTM = duration_auc_reError(50, 10, 2, 10,"LSTM_test_ten_Stages_two_Layer.csv","LSTM_runtime_10_2.csv","LSTM")


CHAY CHO 2 LAYERS


In [ ]:
duration_9_LSTM, auc_9_LSTM, Reconstruction_Error_9_LSTM = duration_auc_reError(50, 10, 2, 9,"LSTM_test_nine_Stages_two_Layer.csv","LSTM_runtime_9_2.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [41]:
duration_10_GRU, auc_10_GRU, Reconstruction_Error_10_GRU = duration_auc_reError(50, 10, 2, 10,"GRU_test_ten_Stages_two_Layer.csv","GRU_runtime_10_2.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [42]:
duration_10_LSTM, auc_10_LSTM, Reconstruction_Error_10_LSTM = duration_auc_reError(50, 10, 2, 10,"LSTM_test_ten_Stages_two_Layer.csv","LSTM_runtime_10_2.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [ ]:
duration_11_GRU, auc_11_GRU, Reconstruction_Error_11_GRU = duration_auc_reError(50, 10, 11,"GRU_test_eleven_batch.csv","GRU_runtime_11.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [ ]:
duration_11_LSTM, auc_11_LSTM, Reconstruction_Error_11_LSTM = duration_auc_reError(50, 10, 11,"LSTM_test_eleven_batch.csv","LSTM_runtime_11.csv","LSTM")


In [45]:
duration_12_GRU, auc_12_GRU, Reconstruction_Error_12_GRU = duration_auc_reError(50, 10, 12,"GRU_test_twelve_batch.csv","GRU_runtime_12.csv","GRU")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [46]:
duration_12_LSTM, auc_12_LSTM, Reconstruction_Error_12_LSTM = duration_auc_reError(50, 10, 12,"LSTM_test_twelve_batch.csv","LSTM_runtime_12.csv","LSTM")


/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/lamhuutoan.nguyen/anaconda3/envs/myenv39/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1, 38, 1])) that is different to the input size (torch.Size([38, 1])). This wil

In [ ]:

plot_boxplot(auc_1)

In [ ]:
duration_2, auc_2, Reconstruction_Error_2 = duration_auc_reError(50, 10, 2,"test_two_batch.csv","runtime_2")


In [ ]:
plot_boxplot(auc_2)

In [ ]:
duration_3, auc_3, Reconstruction_Error_3 = duration_auc_reError(50, 10, 3,"test_three_batch.csv")


In [ ]:
auc_3 = load_auc("test_result_Batch_50/test_three_batch.csv")
auc_melted = auc_3.melt(var_name='Model', value_name='AUC')

In [ ]:
auc_melted

In [ ]:
auc_3 = load_auc("test_three_batch.csv")
plot_boxplot(auc_3)

In [ ]:
duration_4, auc_4, Reconstruction_Error_4 = duration_auc_reError(50, 10, 4,"test_four_batch.csv")


In [ ]:
auc_4 = load_auc("test_four_batch.csv")
plot_boxplot(auc_4)

In [ ]:
duration_5, auc_5, Reconstruction_Error_5 = duration_auc_reError(50, 10, 5,"test_five_batch.csv")


In [ ]:
auc_5 = load_auc("test_five_batch.csv")
plot_boxplot(auc_5)

In [ ]:
duration_6, auc_6, Reconstruction_Error_6 = duration_auc_reError(50, 10, 6,"test_six_batch.csv")


In [ ]:
plot_boxplot(auc_6)

In [ ]:
duration_7, auc_7, Reconstruction_Error_7 = duration_auc_reError(50, 10, 7,"test_seven_batch.csv")


In [ ]:
auc_7 = load_auc("test_seven_batch.csv")
plot_boxplot(auc_7)

In [ ]:
duration_8, auc_8, Reconstruction_Error_8 = duration_auc_reError(50, 10, 8,"test_ten_batch.csv")


In [ ]:
plot_boxplot(auc_8)

In [ ]:
duration_9, auc_9, Reconstruction_Error_9 = duration_auc_reError(50, 10, 9,"test_nine_batch.csv")


In [ ]:
plot_boxplot(auc_9)

In [ ]:
duration_10, auc_10, Reconstruction_Error_10 = duration_auc_reError(50, 10, 10,"test_ten_batch.csv","runtime_10.csv")


In [ ]:
plot_boxplot(auc_10)

In [ ]:
with open("runtime.csv", 'a', newline='') as file:
            file.write(f"{str(duration)}\n")

## Evaluation

Using the threshold, we can turn the problem into a simple binary classification task:

- If the reconstruction loss for an example is below the threshold, we'll classify it as a *normal* heartbeat
- Alternatively, if the loss is higher than the threshold, we'll classify it as an anomaly

Base on the Reconstruction Errors, we can fit the Normal Distribution in it and than find the threshold 

In [ ]:
import scipy as sp
from scipy import stats
def threshold_set(Reconstruction_Error_df):
    threshold = {"GRU" : [],
                 "LSTM" :[]}
    mu_GRU, sigma_GRU = stats.norm.fit(Reconstruction_Error_df["GRU"])
    threshold["GRU"] = mu_GRU + 3 * sigma_GRU
    mu_LSTM, sigma_LSTM = stats.norm.fit(Reconstruction_Error_df["LSTM"])
    threshold["LSTM"] = mu_LSTM + 3 * sigma_LSTM

    return threshold


After having the threshold, we will create the final model that we wana use for the new datas to detect if that is anomaly or not

In [ ]:
def final_model(type, n_epochs, n_stages):
    S_N_dataset, seq_len, n_features, V_N_dataset, V1_dataset, T_dataset, V1_target_df, T_target_df = create_data(10000)  
    model = create_model(type,seq_len, n_features, n_stages)
    final_model = trained_model(model, S_N_dataset, V_N_dataset,n_epochs)    
    
    return model, T_dataset, T_target_df